In [1]:
import numpy as np
import pandas as pd

In [5]:
varE = "Rendimiento tn/ha"
df = pd.DataFrame( {varE:["0-2","2-4","4-6","6-8","8-10"], "f":[11,23,6,8,2] })
df["F"] = df["f"].cumsum()
df[ ["Li","Ls"]] = df[varE].str.split("-",expand=True).astype(int)
df["A"] = df["Ls"] - df["Li"]
df["Pm"] = (df["Ls"] + df["Li"]) / 2 

print(df)

  Rendimiento tn/ha   f   F  Li  Ls  A   Pm
0               0-2  11  11   0   2  2  1.0
1               2-4  23  34   2   4  2  3.0
2               4-6   6  40   4   6  2  5.0
3               6-8   8  48   6   8  2  7.0
4              8-10   2  50   8  10  2  9.0


In [10]:
promedio = (df["f"] * df["Pm"]).sum() / df["f"].sum()
varianza = (df["f"] * ((df["Pm"] - promedio) ** 2)).sum() / (df["f"].sum() - 1)
desviacion = varianza**0.5
Ca = desviacion / promedio
estado = "homogeneos" if Ca < 0.20 else "heterogeneos"

print(f"El promedio es {promedio}")
print(f"La varianza es {varianza}")
print(f"La desviacion es {round(desviacion,4)} tn/ha")
print(f"El coeficiente de variacion es {round(Ca*100,4)}%")
print(f"Los datos son {estado}")

El promedio es 3.68
La varianza es 4.997551020408163
La desviacion es 2.2355 tn/ha
El coeficiente de variacion es 60.7478%
Los datos son heterogeneos


In [14]:
indiceClaseModal = df["f"].idxmax()
claseAnterior = df.loc[indiceClaseModal-1]
claseModal = df.loc[indiceClaseModal]
claseSiguiente = df.loc[indiceClaseModal+1]

moda =claseModal["Li"] + claseModal["A"] * (claseModal["f"] - claseAnterior["f"]) / ((claseModal["f"] -
                                                  claseAnterior["f"]) + (claseModal["f"] - claseSiguiente["f"]))

print(f"La moda es {round(moda,4)} tn/ha")

La moda es 2.8276 tn/ha


In [35]:
def calcularPercentil(k:int,df:pd.DataFrame):
  frecuencia = (k / 100) * df["f"].sum()
  intervalo_percentil = df[ df["F"] >= frecuencia].iloc[0]

  percentil = intervalo_percentil["Li"] + (
      frecuencia - intervalo_percentil["F"] + intervalo_percentil["f"]) / intervalo_percentil["f"] * intervalo_percentil["A"]

  return percentil

In [38]:
print(f"El porcentaje no superado por el 10% de los campos es {round(calcularPercentil(10,df),4)}")
print(f"El porcentaje superado por el 25% de los campos es {round(calcularPercentil(75,df),4)}")

El porcentaje no superado por el 10% de los campos es 0.9091
El porcentaje superado por el 25% de los campos es 5.1667


In [39]:
def calcularInverso(valor, df: pd.DataFrame):
    # Buscar el intervalo
    ia = None
    ie = None
    for index, row in df.iterrows():
        if row['Li'] <= valor <= row['Ls']:
            ie = row
            if index != 0:
              ia = df.loc[index-1]['F']
            elif index == 0:
                ia = 0
            break
    final = (valor - ie['Li']) / \
        (ie['Ls'] - ie['Li'])
    final = final * ie['f']
    final = final + ia
    final = final * 100
    final = final / df['F'].max()
    return final

In [42]:
print(f"El {round(100-calcularInverso(4.7,df),4)}% tiene un rendimiento superior a 4.7 tn/ha.")

El 27.8% tiene un rendimiento superior a 4.7 tn/ha.


In [44]:
asimetria = "asimetria positiva" if promedio > moda else "asimetria negativa"

print("El rendimiento posee una",asimetria, "por lo que podemos decir que es incorrecto")

El rendimiento posee una asimetria positiva por lo que podemos decir que es incorrecto
